In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cd D:\Downloads\emotion_kaggle

In [ ]:
train_text=pd.read_csv("train.txt",header=None,index_col=False,names=['text'])
test_text=pd.read_csv('test.txt',header=None,index_col=False,names=['text'])
val_text=pd.read_csv("val.txt",header=None,index_col=False,names=['text'])

In [ ]:
train_text['text']=train_text['text'].str.split(";")
val_text['text']=val_text['text'].str.split(";")
test_text['text']=test_text['text'].str.split(";")

In [ ]:
train_text

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from nltk.stem import PorterStemmer, WordNetLemmatizer
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
nltk.download('punkt')

In [ ]:
train_text['emotion']=train_text['text'].str[1]
train_text['text']=train_text['text'].str[0]
len(train_text['emotion'].unique())

In [ ]:
val_text['emotion']=val_text['text'].str[1]
val_text['text']=val_text['text'].str[0]
val_text

In [ ]:
test_text['emotion']=test_text['text'].str[1]
test_text['text']=test_text['text'].str[0]
test_text

In [ ]:
final_text=[]
ps=PorterStemmer()
for i in range(len(train_text)):
    review=re.sub('[^a-zA-Z]',' ',train_text['text'][i])
    review=re.split(' ',review)
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    final_text.append(review)
    

In [ ]:
finalval_text=[]
ps=PorterStemmer()
for i in range(len(val_text)):
    review=re.sub('[^a-zA-Z]',' ',val_text['text'][i])
    review=re.split(' ',review)
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    finalval_text.append(review)

In [ ]:
finaltest_text=[]
ps=PorterStemmer()
for i in range(len(test_text)):
    review=re.sub('[^a-zA-Z]',' ',test_text['text'][i])
    review=re.split(' ',review)
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    finaltest_text.append(review)

In [ ]:
train_text['final_text']=final_text
test_text['finaltest_text']=finaltest_text
val_text['finalval_text']=finalval_text

In [ ]:
voc_size=10000
onehot_train=[one_hot(words,voc_size) for words in final_text]

In [ ]:
onehot_test=[one_hot(words,voc_size) for words in finaltest_text]
onehot_val=[one_hot(words,voc_size) for words in finalval_text]

In [ ]:
sent_length=20
embedded_docs_train=pad_sequences(onehot_train,padding='pre',maxlen=sent_length)

In [ ]:
sent_length=20
embedded_docs_test=pad_sequences(onehot_test,padding='pre',maxlen=sent_length)
embedded_docs_val=pad_sequences(onehot_val,padding='pre',maxlen=sent_length)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
embedding_vector_features=300
model=Sequential()
model.add(Embedding(voc_size,300,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(6,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ohe=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')


In [ ]:
X_train=embedded_docs_train.astype(np.float32)
X_test=embedded_docs_test.astype(np.float32)
X_val=embedded_docs_val.astype(np.float32)

In [ ]:
Y_train=pd.DataFrame(ohe.fit_transform(train_text)).iloc[:,:-2].values.astype(np.float32)
Y_test=pd.DataFrame(ohe.fit_transform(test_text)).iloc[:,:-2].values.astype(np.float32)
Y_val=pd.DataFrame(ohe.fit_transform(val_text)).iloc[:,:-2].values.astype(np.float32)

In [ ]:
Y_train

In [ ]:
model.fit(X_train,Y_train, batch_size=128, epochs=5, validation_data=(X_val, Y_val))

In [ ]:
pred=model.predict(X_test)
pred=pred.argmax(axis=-1)
predl=np.zeros((Y_test.shape))
for i in range(len(pred)):
    predl[i][pred[i]]=1
predl

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(Y_test,predl)
score